In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image

from model import build_model
from load_data import get_patches, data_generate

PATCH_SHAPE = (224, 224, 3)

clinical_data = pd.read_csv("data/train.csv")

bag_names = list(clinical_data["ID"])
labels = list(clinical_data["N_category"])
patch_bags = get_patches("data/train_imgs_patch")

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train_bag_names, val_bag_names, train_y, val_y, train_bags, val_bags = train_test_split(bag_names, labels, patch_bags)

In [10]:
train_dataset = tf.data.Dataset.from_generator(
    generator=data_generate,
    output_types=(tf.float32, tf.float32),
    output_shapes=(tf.TensorShape([224, 224, 3]), tf.TensorShape([])),
    args=(train_bag_names, train_y, train_bags)
)

val_dataset = tf.data.Dataset.from_generator(
    generator=data_generate,
    output_types=(tf.float32, tf.float32),
    output_shapes=(tf.TensorShape([224, 224, 3]), tf.TensorShape([])),
    args=(val_bag_names, val_y, val_bags)
)


train_dataset = train_dataset.batch(100)
val_dataset = val_dataset.batch(5)

In [11]:
model = build_model(PATCH_SHAPE)

In [12]:
from loss import bag_loss, bag_accuracy
model.compile(
    optimizer=tf.keras.optimizers.Adam(), loss=bag_loss, metrics=[bag_accuracy]
)

In [13]:
model.fit(
    train_dataset,
    validation_data=val_dataset
)

    104/Unknown - 396s 4s/step - loss: 4.4619 - bag_accuracy: 0.6635